In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.utils import check_random_state
from Fungsi.FungsiAktivasi import FungsiAktivasi
from Fungsi.FungsiLoss import FungsiLoss
from Model.FFNN import FFNN
from Fungsi.FungsiAktivasi import FungsiAktivasi, RELU, SIGMOID, LINEAR, TANH, SOFTMAX
from Fungsi.FungsiLoss import FungsiLoss, CCE, MSE, BCE

def run_test(
    n_hidden=1,
    hidden_size=1,
    epochs=10,
    weight_init_method="he",
    activation_hidden = SIGMOID,
    batch_size=10,
    learning_rate=0.01,
    lambda_L1=1e-4,
    lambda_L2=1e-4,
    seed=42
):
    X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
    random_state = check_random_state(seed)
    permutation = random_state.permutation(X.shape[0])
    X, y = X[permutation], y[permutation]

    X = X.reshape((X.shape[0], -1))
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, train_size=5000, test_size=10000, random_state=seed)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=seed)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    lb = LabelBinarizer()
    y_train_onehot = lb.fit_transform(y_train)
    y_val_onehot = lb.transform(y_val)
    y_test_onehot = lb.transform(y_test)

    input_size = X.shape[1]
    output_size = y_train_onehot.shape[1]

    activation_funcs = [LINEAR] + [activation_hidden for _ in range(n_hidden)] + [SOFTMAX]
    weight_inits = [weight_init_method] * (n_hidden + 2)

    ffnn = FFNN(
        X_train, y_train_onehot,
        input_size = input_size,
        hidden_size = hidden_size,
        output_size = output_size,
        n_hidden = n_hidden,
        batch_size = batch_size,
        learning_rate = learning_rate,
        epoch = epochs,
        activation_func = activation_funcs,
        loss_func = CCE,
        weight_init_method = weight_inits,
        lower_bound = -0.1,
        upper_bound = 0.1,
        mean = 0, variance = 0.1,
        seed = seed,
        lambda_L1 = lambda_L1,
        lambda_L2 = lambda_L2
    )

    history = ffnn.train(
        X_train, y_train_onehot,
        X_val=X_val, y_val=y_val_onehot,
        verbose=1
    )

    plt.figure(figsize=(10, 5))
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training vs Validation Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    def accuracy(y_true, y_pred):
        return np.mean(np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1))

    train_pred = ffnn.predict(X_train)
    test_pred = ffnn.predict(X_test)

    print(f"Train accuracy: {accuracy(y_train_onehot, train_pred):.4f}")
    print(f"Test accuracy: {accuracy(y_test_onehot, test_pred):.4f}")

    return ffnn, history

## Pengaruh Depth dan Width

Depth = n_hidden  
Width = hidden_size

In [14]:
model, hist = run_test(n_hidden = 1, hidden_size = 1)

ValueError: <Fungsi.FungsiAktivasi.FungsiAktivasi object at 0x000001D44201D250> bukan merupakan nama fungsi aktivasi yang valid.

In [ ]:
model, hist = run_test(n_hidden = 1, hidden_size = 2)

In [ ]:
model, hist = run_test(n_hidden = 1, hidden_size = 3)

In [ ]:
model, hist = run_test(n_hidden = 2, hidden_size = 2)

In [ ]:
model, hist = run_test(n_hidden = 3, hidden_size = 2)

In [ ]:
model, hist = run_test(n_hidden = 4, hidden_size = 2)

## Pengaruh fungsi aktivasi

In [ ]:
model, hist = run_test(n_hidden = 4, hidden_size = 2, activation_hidden = LINEAR)

In [ ]:
model, hist = run_test(n_hidden = 4, hidden_size = 2, activation_hidden = RELU)

In [ ]:
model, hist = run_test(n_hidden = 4, hidden_size = 2, activation_hidden = SIGMOID)

In [ ]:
model, hist = run_test(n_hidden = 4, hidden_size = 2, activation_hidden = TANH)

## Pengaruh learning rate

In [ ]:
model, hist = run_test(activation_hidden = TANH, batch_size = 10, n_hidden = 1, hidden_size = 3, epochs = 100, learning_rate = 0.1)

In [ ]:
model, hist = run_test(activation_hidden = TANH, batch_size = 10, n_hidden = 1, hidden_size = 3, epochs = 100, learning_rate = 0.01)

In [ ]:
model, hist = run_test(activation_hidden = TANH, batch_size = 10, n_hidden = 1, hidden_size = 3, epochs = 100, learning_rate = 0.001)

## Pengaruh inisialisasi bobot

In [ ]:
model, hist = run_test(weight_init_method = "zero")

In [ ]:
model, hist = run_test(weight_init_method = "uniform")

In [ ]:
model, hist = run_test(weight_init_method = "normal")

## Pengaruh regularisasi

In [ ]:
model, hist = run_test(lambda_L1 = 0, lambda_L2 = 0)

In [ ]:
model, hist = run_test(lambda_L1 = 1e-4, lambda_L2 = 0)

In [ ]:
model, hist = run_test(lambda_L1 = 0, lambda_L2 = 1e-4)